In [1]:
import pdfplumber
import pandas as pd
import datetime
import os
import re
import numpy as np

In [2]:
path = 'data/raw/'
filenames = os.listdir(path)
filenames = sorted(filenames)

In [3]:
unt_pages = [*range(129,136), *range(140,146), *range(152,156), *range(215,285), 288, 289, 291, 292, 293, *range(294,333), *range(334, 899), *range(923,947), *range(948,1207), *range(1220,1618), *range(1619,1651), *range(1653,1872), *range(1873,2121), *range(2124,2132), *range(2138, 2575), *range(2577, 2593), *range(2596, 2613), *range(2618, 2695), *range(2701, 2724), *range(2730, 2764), *range(2775,2783), 2788, 2792, 2793, 2808, 2809, 2812, 2814, 2823, 2824, 2825, 2847, 2848, 2850, 2851, 2856, 2858, 2862, 2866, 2867, 2868, 2869, 2870, 2871, 2873, 2874, 2875, 2876, 2877, 2878, 2879, 2880, 2881, 2882, 2883, 2885, 2886, 2887, 2888, 2889, 2890, 2891, 2893, 2894, 2895, 2896, 2897, 2898, 2899, 2900, 2902, 2904, *range(2906,2930), *range(2932, 2943), *range(2944,2954), *range(2959,2965), *range(2967,2991)]

In [4]:
filenames.pop(0)

'.DS_Store'

In [5]:
counter = 0
ds = []
pass_nums = []
fail_nums = []
#loop through each file
for filename in filenames:

    #add the file name to the path
    filepath = path + filename

    #open the file with pdfplumber
    with pdfplumber.open(filepath) as pdf:

            #create empty string to store pdf text
            pdf_text = ''

            #loop through each page in the pdf and add the text to the pdf_text string
            if 'unt' not in filename:
                for i in range(0, len(pdf.pages)):
                    page = pdf.pages[i]
                    text = page.extract_text()
                    pdf_text = pdf_text + '\n' + text
                
            else: #dont want to loop through all 4k plus pages of UNT doc=
                for i in unt_pages: 
                    page = pdf.pages[i-1] #i-1 because page indicies start at 0 and I wrote down actual page numbers ie. [0] is page 1
                    text = page.extract_text()
                    pdf_text = pdf_text + '\n' + text
                unt_text = pdf_text #save the text for later
            
            if re.findall('Alert\s{1,2}#(.*)\s{1,2}(was|matched)', pdf_text) != []: #grab all the alert numbers for alert notifications and topic rollups
                    ids = re.findall('Alert\s{1,2}#(.*)\s{1,2}(was|matched)', pdf_text)
                    
                    #get only the alert number not anything else, including whitespaces
                    for i,j in enumerate(ids):
                        ids[i] = j[0][:9].strip() 

            elif re.findall('Alert ID: (.*) \n', pdf_text) != []: #grab all the alert numbers
                ids = re.findall('Alert ID: (.*) \n', pdf_text)

            else: #set ids to empty list if no alert numbers are found
                ids = []

            #split the text into a list of lines called y
            y = pdf_text.split('\n')
                
            #loop through each id it finds
            for i in range(0, len(ids)):
                d = {}
                   
                d['alert_id'] = ids[i]
                d['filename'] = filename

                #if it is not the last id in the list, grab the part of y from that id to the line that has the next id in it
                id1 = ids[i]
                idx1 = [idx for idx, s in enumerate(y) if id1 in s][0]
                
                try:
                    id2 = ids[i+1]
                    idx2 = [idx for idx, s in enumerate(y) if id2 in s][0]
                    z = y[idx1:idx2]
                
                except: #should be the case for the last id in the list
                    z = y[idx1:]

                chunk = ''
                    
                #loop through the subsection of y and add the text to the chunk string
                for line in z:
                    chunk += line + '\n'
                
                #time posted
                if re.findall('Time\s{1,2}posted:(.*)', chunk) != []: #topic rollup and alert notification
                    d['time_posted'] = re.findall('Time\s{1,2}posted:(.*)', chunk)[0]
                
                elif re.findall('Published:(.*)', chunk) != []: #alert summary
                    d['time_posted'] = re.findall('Published:(.*)', chunk)[0]
                
                else: #failure case
                    d['time_posted'] = ''
                
                #locations
                if re.findall('Locations: (.*)', chunk) != []: #alert notification and topic rollup
                    d['locations'] = re.findall('Locations: (.*)', chunk)[0]
                
                elif re.findall('Location\(s\):(.*)', chunk) != []: #alert summary
                    d['locations'] = re.findall('Location\(s\):(.*)', chunk)[0]
                
                else: #failure case
                    d['locations'] = ''
                
                #link
                if re.findall('Details:(.*)', chunk) != []: #alert summary
                    d['link'] = re.findall('Details:(.*)', chunk)[0]
                
                elif re.findall('Link:(.*)', chunk) != []: #topic rollup
                    d['link'] = re.findall('Link:(.*)', chunk)[0]
                
                else: #failure case
                    d['link'] = ''

                #source
                if re.findall('Source: (.*)', chunk) != []: 
                    d['source'] = re.findall('Source: (.*)', chunk)[0]
                
                else:
                    d['source'] = ''

                #author
                if re.findall('Author: (.*)', chunk) != []: #alert summary and topic rollup
                    d['author'] = re.findall('Author: (.*)', chunk)[0]
                
                
                if re.findall('Post\s{1,2}from(.*)', chunk) != []: #alert notification
                    
                    pf = re.findall('Post\s{1,2}from(.*)', chunk)[0]
                    pf = pf.strip()
                    
                   #if we don't already have an author, set the author to the post from value
                    try: 
                        d['author']
                    except KeyError:
                        d['author'] = pf

                elif re.findall('(.*) posted\s{1,2}an\s{1,2}image', chunk) != []: #UNT alert notification for Instagram
                    pf = re.findall('(.*) posted\s{1,2}an\s{1,2}image', chunk)[0]
                    
                    #if we dont already have a source set the source to what it says in the post from line
                    try:
                        d['source']
                    except KeyError:
                        d['source'] = 'Instagram'
                    
                    #if we don't already have an author, set the author to the post from value
                    try:
                        d['author']
                    except KeyError:
                        d['author'] = pf
                
                elif re.findall('(.*)posted\s{1,2}a\s{1,2}video', chunk) != []: #UNT alert notification
                    pf = re.findall('(.*)posted\s{1,2}a\s{1,2}video', chunk)[0]
                    
                    try:
                        d['author']
                    except KeyError:
                        d['author'] = pf
                
                elif re.findall('(.*)posted\s{1,2}a\s{1,2}note', chunk) != []: #UNT alert notification for Google+
                    pf = re.findall('(.*)posted\s{1,2}a\s{1,2}note', chunk)[0]
                    d['source'] = 'Google+'
                    
                    try:
                        d['author']
                    except KeyError:
                        d['author'] = pf
            
                elif re.findall('(.*)posted\s{1,2}a\s{1,2}bookmark', chunk) != []: #UNT alert notification for Google+
                    pf = re.findall('(.*)posted\s{1,2}a\s{1,2}bookmark', chunk)[0]
                    d['source'] = 'Google+'
                    
                    try:
                        d['author']
                    except KeyError:
                        d['author'] = pf
                
                elif re.findall('(.*)shared\s{1,2}an\s{1,2}activity', chunk) != []: #UNT alert notification for Google+
                    pf = re.findall('(.*)shared\s{1,2}an\s{1,2}activity', chunk)[0]
                    d['source'] = 'Google+'
                    
                    try:
                        d['author']
                    
                    except KeyError:
                        d['author'] = pf

                elif re.findall('(.*)shared\s{1,2}a\s{1,2}photo', chunk) != []: #UNT alert notification for Google+
                    pf = re.findall('(.*)shared\s{1,2}a\s{1,2}photo', chunk)[0]
                    d['source'] = 'Google+'
                    
                    try:
                        d['author']
                    except KeyError:
                        d['author'] = pf
                
                elif re.findall('(.*)shared\s{1,2}a\s{1,2}video', chunk) != []: #UNT alert notification for Google+
                    pf = re.findall('(.*)shared\s{1,2}a\s{1,2}video', chunk)[0]
                    d['source'] = 'Google+'
                    
                    try:
                        d['author']
                    except KeyError:
                        d['author'] = pf 
                
                else:
                    pf = ''

                #delivered to
                if re.findall('delivered to (.*)', chunk) != []:
                    d['delivered_to'] = re.findall('delivered to (.*)', chunk)[0]
                else:
                    d['delivered_to'] = ''
                
                #geodata
                if re.findall('Geodata: (.*)', chunk) != []: #topic rollup
                    d['geodata'] = re.findall('Geodata: (.*)', chunk)[0]
                else: #failure case
                    d['geodata'] = ''
                
                #rmap match
                if re.findall('Match: (.*)', chunk) != []: #topic rollup and alert notification
                    d['rmap_match'] = re.findall('Match: (.*)', chunk)[0]
                else: #failure case
                    d['rmap_match'] = ''
            
                #client id and type
                if 'alert_summary' in filename: #alert summary
                    d['client_id'] = filename[14:18]
                    d['type'] = 'alert summary'
                
                elif 'topic_rollup' in filename: #topic rollup
                    d['client_id'] = filename[13:17]
                    d['type'] = 'topic rollup'
                
                elif 'palm_beach' in filename: #alert notification (palm beach)
                    d['client_id'] = '3408'
                    d['type'] = 'alert notification'
                
                elif 'unt' in filename: #alert notification (unt)
                    d['client_id'] = '1042'
                    d['type'] = 'alert notification'
                
                else: #failure case
                    d['client_id'] = ''
                    d['type'] = ''

                #content
                if 'topic_rollup' in filename: #topic rollup
                    #grab lines from the line with 'Content:' in it to the line with 'Link:' in it then combine them 
                    if [idx for idx, s in enumerate(z) if 'Content:' in s] != []:
                        
                        if [idx for idx, s in enumerate(z) if 'Link' in s]  != []:
                            
                            c_idx = [idx for idx, s in enumerate(z) if 'Content:' in s][0]
                            l_idx = [idx for idx, s in enumerate(z) if 'Link' in s][0]
                            lines = z[c_idx:l_idx]
                            d['content'] = '\n'.join(lines)
                            
                            #clean up the content
                            d['content'] = d['content'].replace('Content: ', '')
                            # d['content'] = d['content'].replace('\n', ' ')
                        
                        else:
                            
                            c_idx = [idx for idx, s in enumerate(z) if 'Content:' in s][0]
                            lines = z[c_idx:]
                            d['content'] = '\n'.join(lines)
                            
                            #clean up the content
                            d['content'] = d['content'].replace('Content: ', '')
                            # d['content'] = d['content'].replace('\n', ' ')
                    
                    else:
                        d['content'] = chunk
                
                elif 'palm_beach' in filename:
                    
                    if [idx for idx, s in enumerate(z) if 'Post from' in s] != []:
                        
                        if [idx for idx, s in enumerate(z) if 'View Alert' in s] != []:
                            
                            #grab lines from the line after 'Post from' in it to the line with 'View Alert' in it then combine them
                            pf_idx = [idx for idx, s in enumerate(z) if 'Post from' in s][0]
                            va_idx = [idx for idx, s in enumerate(z) if 'View Alert' in s][0]
                            lines = z[pf_idx+1:va_idx]
                            
                            #clean up the content
                            d['content'] = '\n'.join(lines)
                            # d['content'] = d['content'].replace('\n', ' ')
                        
                        else:
                            pf_idx = [idx for idx, s in enumerate(z) if 'Post from' in s][0]
                            lines = z[pf_idx+1:]
                            d['content'] = '\n'.join(lines)
                            # d['content'] = d['content'].replace('\n', ' ')
                    
                    else:
                        d['content'] = chunk
                
                elif 'unt' in filename:
                    
                    if pf != '':
                        
                        #grab lines from the line after 'Post from' in it to the line with 'View Alert' in it then combine them
                        pf_idx = [idx for idx, s in enumerate(z) if pf in s][0]
                        
                        if re.findall('Social\s{1,2}Sentinel\s{1,2}Alert\s{1,2}Notification', chunk) != []:
                            va = re.findall('Social\s{1,2}Sentinel\s{1,2}Alert\s{1,2}Notification', chunk)[0]
                            va_idx = [idx for idx, s in enumerate(z) if va in s][0]
                            lines = z[pf_idx+1:va_idx]
                            #clean up the content
                            d['content'] = '\n'.join(lines)
                            # d['content'] = d['content'].replace('\n', ' ')
                            if ids[i] not in pass_nums:
                                pass_nums.append(ids[i])
                        else:
                            lines = z[pf_idx+1:]   
                            d['content'] = '\n'.join(lines)
                            if ids[i] not in pass_nums:
                                pass_nums.append(ids[i])
                            # d['content'] = d['content'].replace('\n', ' ')
                            
                    else: #pf == ''
                        #failure case: write the whole chunk as content
                        d['content'] = chunk
                        fail_nums.append(ids[i])
                        
                elif 'alert_summary' in filename:
                    #grab lines from the line after 'Author:' in it to the end of the chunk then combine them
                    a_idx = [idx for idx, s in enumerate(z) if 'Author:' in s][0]
                    if 'All Rights Reserved.' in chunk:
                        ar_idx = [idx for idx, s in enumerate(z) if 'All Rights Reserved.' in s][0]
                        lines = z[a_idx+1:ar_idx]
                    else:
                        lines = z[a_idx+1:]
                    #clean up the content
                    d['content'] = '\n'.join(lines)
                    d['content'] = d['content'].replace('\n', ' ')
                else:
                    #failure case: write the whole chunk as content
                    d['content'] = chunk

        
                ds.append(d)

In [6]:
#check how many alert just failed completely
fails = list(set([i for i in fail_nums if i not in pass_nums]))
print(len(fails))

47


In [29]:
df = pd.DataFrame(ds)

In [30]:
map_dict = {'2193': 'Palm Beach State College', 
 '3408':'Palm Beach State College',
 '3696':'Gulf Coast State College',
 '3790':'Gulf Coast State College',
 '4896':'Indian River State College',
 '3792':'Gulf Coast State College',
 '1042':'University of North Texas'}

schools = []
for i in list(df['client_id']):
    try:
        schools.append(map_dict[i])
    except:
        schools.append('')
df['school'] = schools

In [31]:
df['datetime'] = pd.to_datetime(df['time_posted'], errors='coerce')


In [32]:
print(len(df))
df= df.loc[df['content'].str.strip() != ''].reset_index(drop=True)
print(len(df))
df = df.groupby('alert_id').first().reset_index() #dedupe with alert_id
print(len(df))
df = df.loc[df['content'].str.len() < 25000].reset_index() #get rid of one outlier screed
print(len(df))

6979
5921
4261
4260


In [33]:
df.to_csv('data/created/flagged_tweets.csv', index=False)